In [ ]:
# Lets start with API
# Twitter example no fun
# Libraries no fun
# Let's go to Mashape!

In [ ]:
import requests

# Docs https://market.mashape.com/omgvamp/hearthstone
MASHAPE_API_URL = 'https://omgvamp-hearthstone-v1.p.mashape.com/'
# Don't forget to fill this key with your registration data
MASHAPE_API_KEY = ''

In [ ]:
auth_header = {'X-Mashape-Key': MASHAPE_API_KEY}
response = requests.get(MASHAPE_API_URL+'/cards', headers=auth_header)
data = response.json()

In [ ]:
len(response.content)

In [ ]:
response = requests.get(MASHAPE_API_URL+'/cards?collectible=1', headers=auth_header)
collection_data = response.json()
len(response.content)

In [ ]:
collection_names = data.keys()
collection_names

In [ ]:
data['Credits']

In [ ]:
from IPython.display import Image
Image(data['Credits'][5]['img'])

In [ ]:
# Now let's go to some hacking.
# http://www.hearthpwn.com/decks/719434-one-turn-kill-math-hunter-antonamby

full_url = 'http://www.hearthpwn.com/decks/719434-one-turn-kill-math-hunter-antonamby'
page_data = requests.get(full_url)

In [ ]:
len(page_data.content)

In [ ]:
trimmed_url = 'http://www.hearthpwn.com/decks/719434'
page_data = requests.get(full_url)
len(page_data.content)

In [ ]:
ajax_url = 'http://www.hearthpwn.com/decks/listing/719434/class'
page_data = requests.get(ajax_url)
len(page_data.content)

In [ ]:
page_data.content

In [ ]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import bs4
source = bs4.BeautifulSoup(page_data.content, 'html.parser')
print source

In [ ]:
print source.prettify()

In [ ]:
links = source.find_all('a')
for link in links:
    print link.string.strip()

In [ ]:
links = [link for link in source.find_all('a') if 'data-id' in link.attrs]
for link in links:
    print link.string.strip()

In [ ]:
deck_data = {}
for link in links:
    text = link.parent.parent.text.strip()
    card, card_num = text.split(u'×')
    deck_data[card.strip()] = int(card_num.strip())
deck_data

In [ ]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
deck = pd.Series(deck_data)
deck.plot(kind='bar')

In [ ]:
from collections import defaultdict
rarity_dict = defaultdict(int)
for link in links:
    text = link.parent.parent.text.strip()
    rarity = link.attrs['data-rarity']
    _, card_num = text.split(u'×')
    card_num = int(card_num.strip())
    rarity_dict[rarity] += card_num
rarity_dict

In [ ]:
rarity_s = pd.Series(rarity_dict)
rarity_s.plot(kind='pie')

In [ ]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.user', 
    sep='|', names=u_cols)

users.head()

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
    sep='\t', names=r_cols)

ratings.head()

In [ ]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']

movies = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.item', 
    sep='|', names=m_cols, usecols=range(5))

movies.head()

In [ ]:
movies.dtypes

In [ ]:
users['sex'].describe()

In [ ]:
sex_series = users['sex']
sex_series.value_counts().plot(kind='bar')

In [ ]:
users['age'].hist()

In [ ]:
females = users[users['sex'] == 'F']
females['age'].hist()

In [ ]:
males = users[users['sex'] == 'M']
males['age'].hist()

In [ ]:
users['sex'] == 'M'

In [ ]:
users.age.mean()

In [ ]:
print males.age.mean()
print females.age.mean()

In [ ]:
## split data
grouped_data = ratings['rating'].groupby(ratings['movie_id'])
## average and combine
average_ratings = grouped_data.mean()
print "Average ratings:"
print average_ratings.head()

print 'Votes for movies'
print grouped_data.count().head()

maximum_rating = average_ratings.max()
print 'Maximum rating: %f' % maximum_rating
good_movie_ids = average_ratings[average_ratings == maximum_rating].index

print "Best movie titles"
print movies[movies.movie_id.isin(good_movie_ids)].title
print